In [1]:
from census import Census
from us import states
import pandas as pd
import geopandas as gpd
import json
from pandas import DataFrame
import censusdata

In [2]:
f = open('config/keys.json')
data = json.load(f)
acs_api_key = data['acs']['key']
c = Census(acs_api_key)

In [26]:
# geo includes tract and blockgroup
def get_ACS5_data(tablename, fields, geo, year):
    if fields == "*":
        if year > 2019:
            YEAR = 2019
        else:
            YEAR = year
        fields = [key for key, value in censusdata.censustable('acs5', YEAR, tablename).items()]
    
    if tablename[0] == 'S':
        func_nm = 'acs5st'
    elif tablename[0] == 'D':
        func_nm = 'acs5dp'
    else:
        func_nm = 'acs5'
    
    ctycode = "039"
    al = "*"
  
    data = DataFrame(eval(f'c.{func_nm}.state_county_{geo}(fields = fields, state_fips = states.OR.fips, county_fips = ctycode, {geo} = al,year = year)'))
    
    if geo == "tract":
        data["GEOID"] = data["state"] + data["county"] + data["tract"]
        data = data.drop(columns = ["state", "county", "tract"])
        
    else:
        data["GEOID"] = data["state"] + data["county"] + data["tract"] + data["block group"]
        data = data.drop(columns = ["state", "county", "tract", "block group"])
        
    return(data)

In [10]:
def get_ACS5_years(tablename, fields, geo, years):
    data = pd.DataFrame()
    for year in years:
        ndata = get_ACS5_data(tablename = tablename, 
                             fields = fields, 
                             geo = geo, 
                             year = year)
        ndata['Year'] = year
        data = data.append(ndata)
        print(tablename + ' ' + str(year))
    return(data)

In [11]:
lane_pop_tract = get_ACS5_years(tablename = 'B01001', fields = '*', geo = 'tract', years = range(2012, 2021))

B01001 2012
B01001 2013
B01001 2014
B01001 2015
B01001 2016
B01001 2017
B01001 2018
B01001 2019
B01001 2020


In [31]:
censusdata.printtable(censusdata.censustable('acs5', 2019, 'S1901'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
S1901_C01_001E | INCOME IN THE PAST 12 MONTHS ( | !! !! Estimate Households Total                          | int  
S1901_C01_002E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households Total Less than $10,000     | float
S1901_C01_003E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households Total $10,000 to $14,999    | float
S1901_C01_004E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households Total $15,000 to $24,999    | float
S1901_C01_005E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households Total $25,000 to $34,999    | float
S1901_C01_006E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households Total $35,000 to $49,999    | float
S1901_C01_007E | INCOME IN THE PAST 12 MONTHS ( | !! !! !! Estimate Households To

In [ ]:
print(get_ACS5_data(tablename='S1901', fields=['S1901_C01_001E', '', ''], geo, year).head(5))